In [1]:
# Relevant libraries and functions
from __future__ import print_function
import random
import matplotlib.pyplot as plt
import scipy.ndimage
import numpy as np, h5py
import os, time, sys
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import BatchNormalization, Convolution2D, Input, SpatialDropout2D, UpSampling2D, MaxPooling2D, concatenate
from keras.layers.core import Activation
from keras.layers import Dense, Dropout, Conv1D, Input, Conv2D, add, Conv3D, Reshape
from keras.callbacks import History, EarlyStopping, ModelCheckpoint, CSVLogger
from itertools import cycle
from sklearn import metrics
from keras.optimizers import RMSprop
from keras.utils import np_utils
from tensorflow.compat.v1.keras.backend import set_session
from keras.layers.convolutional import Convolution2D, MaxPooling2D, SeparableConv2D, Conv2DTranspose
import matplotlib.pyplot as pyplot


In [2]:
f_data = './trainbi'
#f_data = '/gs/home/wenjiao/zrx-2DGP/Group-bi/pydata/trainbi1' # Directory with trainging data
stacks = os.listdir(f_data)
numS = int(len(stacks))

nTG = 200 # Number of time-points
xX = 100


tpsfD = np.ndarray(
        (numS, int(nTG), int(xX),  int(1)), dtype=np.float32
        )
t1 = np.ndarray(
        (numS, int(xX), int(1)), dtype=np.float32
        )
t2 = np.ndarray(
        (numS, int(xX), int(1)), dtype=np.float32
        )
tR = np.ndarray(
        (numS, int(xX),  int(1)), dtype=np.float32
        )

i = 0;
for d in stacks:
    # Save values to respective mapping
    f = h5py.File(os.path.join(f_data,d),'r') 
    tpsfD[i,:,:,0] = f.get('sigD')
    f = h5py.File(os.path.join(f_data,d),'r') 
    t1[i,:,0] = f.get('t1')
    f = h5py.File(os.path.join(f_data,d),'r') 
    t2[i,:,0] = f.get('t2')
    f = h5py.File(os.path.join(f_data,d),'r') 
    tR[i,:,0] = f.get('rT')
    i = i + 1

tpsfD[np.isnan(tpsfD)]=0
    
tpsfD =  np.moveaxis(tpsfD, 1, -2)

In [3]:
# Ensure TPSF voxel shape is correct dimensionality (# samples, x, y, time-points, 1)
tpsfD.shape

(300, 100, 200, 1)

In [4]:
# Relevant resblock functions (Keras API)
def resblock_2D(num_filters, size_filter, x):
    Fx = Conv2D(num_filters, size_filter, padding='same', activation=None)(x)
    Fx = Activation('relu')(Fx)
    Fx = Conv2D(num_filters, size_filter, padding='same', activation=None)(Fx)
    output = add([Fx, x])
    output = Activation('relu')(output)
    return output

def resblock_2D_BN(num_filters, size_filter, x):
    Fx = Conv2D(num_filters, size_filter, padding='same', activation=None)(x)
    Fx = BatchNormalization()(Fx) 
    Fx = Activation('relu')(Fx)
    Fx = Conv2D(num_filters, size_filter, padding='same', activation=None)(Fx)
    Fx = BatchNormalization()(Fx)
    output = add([Fx, x])
    #output = BatchNormalization()(output)
    output = Activation('relu')(output)
    return output

def resblock_3D_BN(num_filters, size_filter, x):
    Fx = Conv3D(num_filters, size_filter, padding='same', activation=None)(x)
    Fx = BatchNormalization()(Fx)
    Fx = Activation('relu')(Fx)
    Fx = Conv3D(num_filters, size_filter, padding='same', activation=None)(Fx)
    Fx = BatchNormalization()(Fx)
    output = add([Fx, x])
    output = BatchNormalization()(output)
    output = Activation('relu')(output)
    return output

def xCeptionblock_2D_BN(num_filters, size_filter, x):
    Fx = SeparableConv2D(num_filters, size_filter, padding='same', activation=None)(x)
    Fx = BatchNormalization()(Fx)
    Fx = Activation('relu')(Fx)
    Fx = SeparableConv2D(num_filters, size_filter, padding='same', activation=None)(Fx)
    Fx = BatchNormalization()(Fx)
    output = add([Fx, x])
    output = Activation('relu')(output)
    return output

In [5]:
modelD = None
xX = 100;


t_data = Input(shape=(xX,200,1))
tpsf = t_data

# # # # # # # # 2D-Model # # # # # # # #

tpsf = Conv2D(10,kernel_size=(1,10),strides=(1,5), padding='same', activation=None, data_format="channels_last")(tpsf)
tpsf = BatchNormalization()(tpsf)
tpsf = Activation('relu')(tpsf)


tpsf = Conv2D(256,1, padding='same', activation=None, data_format="channels_last")(tpsf)
tpsf = BatchNormalization()(tpsf)
tpsf = Activation('relu')(tpsf)
tpsf = Conv2D(256,1, padding='same', activation=None, data_format="channels_last")(tpsf)
tpsf = BatchNormalization()(tpsf)
tpsf = Activation('relu')(tpsf)
tpsf = resblock_2D_BN(256, 1,tpsf)
tpsf = resblock_2D_BN(256, 1,tpsf)
tpsf = Reshape((xX,10240))(tpsf)

# Short-lifetime branch
imgT1 = Dense(64, activation=None)(tpsf)
imgT1 = BatchNormalization()(imgT1)
imgT1 = Activation('relu')(imgT1)
imgT1 = Dense(32, activation=None)(imgT1)
imgT1 = BatchNormalization()(imgT1)
imgT1 = Activation('relu')(imgT1)
imgT1 = Dense(1, activation=None)(imgT1)
imgT1 = Activation('relu')(imgT1)
imgT1 = keras.layers.Dense(1, activation='relu',name='T1')(imgT1)
# Long-lifetime branch
imgT2 = Dense(64, activation=None)(tpsf)
imgT2 = BatchNormalization()(imgT2)
imgT2 = Activation('relu')(imgT2)
imgT2 = Dense(32, activation=None)(imgT2)
imgT2 = BatchNormalization()(imgT2)
imgT2 = Activation('relu')(imgT2)
imgT2 = Dense(1, activation=None)(imgT2)
imgT2 = Activation('relu')(imgT2)
imgT2 = keras.layers.Dense(1,activation='relu',name='T2')(imgT2)
# Amplitude-Ratio branch
imgTR = Dense(64, activation=None)(tpsf)
imgTR = BatchNormalization()(imgTR)
imgTR = Activation('relu')(imgTR)
imgTR = Dense(32, activation=None)(imgTR)
imgTR = BatchNormalization()(imgTR)
imgTR = Activation('relu')(imgTR)
imgTR = Dense(1, activation=None)(imgTR) 
imgTR = Activation('relu')(imgTR)
imgTR = keras.layers.Dense(1,activation='relu',name='TR')(imgTR)


In [6]:
modelD = Model(inputs=[t_data], outputs=[imgT1,imgT2,imgTR])

In [7]:
losses = {"T1": "mse","T2": "mse","TR": "mse"}

In [9]:
metric = {"T1": "mae","T2": "mae","TR": "mae"}

In [ ]:
modelD.compile(loss= losses,
              optimizer= 'adam',
              metrics= metric)
# Setting patience (patience = 15 recommended)
earlyStopping = EarlyStopping(monitor='val_loss', 
                              patience = 15, 
                              verbose = 0,
                              mode = 'auto')

fN = 'testName' # Assign some name for weights and training/validation loss curves here

# Save loss curve (mse) and MAE information over all trained epochs. (monitor = '' can be changed to focus on other tau parameters)
modelCheckPoint = ModelCheckpoint(filepath=fN+'.h5', 
                                  monitor='val_loss', 
                                  save_best_only=True, 
                                  verbose=0)
# Train network (80/20 train/validation split, batch_size=20 recommended, nb_epoch may vary based on application)
history = History()
csv_logger = CSVLogger(fN+'.log')
history = modelD.fit([tpsfD], [t1,t2,tR],
          validation_split=0.2,
          batch_size=20, epochs=500, verbose=1, shuffle=True, callbacks=[earlyStopping,csv_logger,modelCheckPoint])

In [10]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# Post-training: load "best" trained weights (obtained through patience - lowest value of loss)
# THIS CAN BE ANY WEIGHT FILE, AS LONG AS THE NETWORK ARCHITECTURE MATCHES THE ONE USED!
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

modelD.load_weights(fN+'.h5')

In [12]:
# Upload test data and use 3D-CNN for inference
t_data = './fitbi'
#t_data = '/gs/home/wenjiao/zrx-2DGP/Group-bi/pydata/fitbi' # directory with test data
stacksT = os.listdir(t_data)
numT = int(len(stacksT))

nTG = 200
xX = 100


tpsfT = np.ndarray(
        (numT, int(nTG), int(xX),  int(1)), dtype=np.float32
        )
t1T = np.ndarray(
        (numT, int(xX), int(1)), dtype=np.float32
        )
t2T = np.ndarray(
        (numT, int(xX), int(1)), dtype=np.float32
        )
tRT = np.ndarray(
        (numT, int(xX),  int(1)), dtype=np.float32
        )

i = 0;
for d in stacksT:
    # Save values to respective mapping
    f = h5py.File(os.path.join(t_data,d),'r') 
    tpsfT[i,:,:,0] = f.get('sigD')
    f = h5py.File(os.path.join(t_data,d),'r') 
    t1T[i,:,0] = f.get('t1')
    f = h5py.File(os.path.join(t_data,d),'r') 
    t2T[i,:,0] = f.get('t2')
    f = h5py.File(os.path.join(t_data,d),'r') 
    tRT[i,:,0] = f.get('rT')
    i = i + 1
    
tpsfT =  np.moveaxis(tpsfT, 1, -2)
# tpsfT = np.moveaxis(tpsfT, 1, 2)

In [13]:
# Perform inference on test data with trained model
testV = modelD.predict(tpsfT)
# Predicted t1 values

import pickle

def save_variable(v,filename):
        f = open(filename,'wb')
        pickle.dump(v,f)
        f.close()
        return filename

def load_variable(filename):
        f=open(filename,'rb')
        r=pickle.load(f)
        f.close()
        return r

import joblib
joblib.dump(testV,'testV1.pkl')


modelD.save('30_2dgp.h5')

modelD.save('30_2dgp.h5')
